In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
data = pd.read_csv("emotions.csv")

# Data preprocessing
X = data['text']
y = data['label']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Padding sequences
max_length = 100  # Define your maximum sequence length
X = pad_sequences(X, maxlen=max_length)

# Convert labels to one-hot encoding
num_classes = len(data['label'].unique())
y = pd.get_dummies(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Predict labels
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(np.array(y_test), axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

Epoch 1/10
8337/8337 [==============================] - 913s 109ms/step - loss: 0.4175 - accuracy: 0.8263 - val_loss: 0.3445 - val_accuracy: 0.8531
Epoch 2/10
8337/8337 [==============================] - 871s 104ms/step - loss: 0.3214 - accuracy: 0.8592 - val_loss: 0.3296 - val_accuracy: 0.8530
Epoch 3/10
8337/8337 [==============================] - 893s 107ms/step - loss: 0.2940 - accuracy: 0.8696 - val_loss: 0.3431 - val_accuracy: 0.8521
Epoch 4/10
8337/8337 [==============================] - 839s 101ms/step - loss: 0.2699 - accuracy: 0.8794 - val_loss: 0.3555 - val_accuracy: 0.8483
Epoch 5/10
2606/2606 [==============================] - 27s 11ms/step - loss: 0.3314 - accuracy: 0.8519
Test Loss: 0.33140361309051514
Test Accuracy: 0.8519469499588013
2606/2606 [==============================] - 27s 10ms/step
Precision: 0.8548735285085367
Recall: 0.8519469302559919
F1-score: 0.8507456873550869
